<a href="https://colab.research.google.com/github/alvarogallardo/Proyecto_EL5206/blob/main/Proyecto_EL5206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import sys
sys.path.append('/content/drive/MyDrive/Proyecto_EL5206') #Dataset 
sys.path.append('/content/drive/MyDrive/Proyecto_EL5206/jpg')

Librerias

In [17]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions


In [18]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import chisquare
from math import sqrt
from keras.applications.vgg16 import VGG16
from google.colab import drive
import sys
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input,decode_predictions

Funciones auxiliares

In [19]:
def get_pixel(img, center, x, y):
    new_value = 0    
    try:
        if img[x][y] >= center:
            new_value = 1            
    except:
        pass   
    return new_value 
# Calcular LBP
def lbp_calculated_pixel(img, x, y):  
    center = img[x][y]
    val_ar = []
    # top_left
    val_ar.append(get_pixel(img, center, x-1, y-1)) 
    # top
    val_ar.append(get_pixel(img, center, x-1, y))   
    # top_right
    val_ar.append(get_pixel(img, center, x-1, y + 1)) 
    # right
    val_ar.append(get_pixel(img, center, x, y + 1))  
    # bottom_right
    val_ar.append(get_pixel(img, center, x + 1, y + 1)) 
    # bottom
    val_ar.append(get_pixel(img, center, x + 1, y))
    # bottom_left
    val_ar.append(get_pixel(img, center, x + 1, y-1))
    # left
    val_ar.append(get_pixel(img, center, x, y-1))    
    # Transformamos de binario a decimal
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0  
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]    
    return val

#Aplicar LBP
def lbp(im):
  img_bgr = cv2.imread(im, 1)
  img_bgr = cv2.resize(img_bgr, (600,500))
  height, width, _ = img_bgr.shape 
  img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
  img_lbp = np.zeros((height, width),np.uint8)
  for i in range(0, height):
    for j in range(0, width):
        img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
  return img_lbp




Tradicional

In [20]:
def handcrafted(im):
  if len(im) == 10:
    im= '/content/drive/MyDrive/Proyecto_EL5206/jpg/'+ str(im)
  im=lbp(im)
  histogramas= []
  k=0
  y=0
  #dividimos las imagenes
  for j in range(6):
    x=0
    for i in range(5):
      histogram,bin_edges = np.histogram(im[x:x+100, y:y+100],bins=84,range=(0,255))
      for element in histogram:
        #Guardamos los valores en un vector de caracteristicas
        histogramas.append(element)
      x+=100
    y+=100
  return histogramas

CNN VGG

In [27]:
def cnn_vgg(img_path, model):
  if len(img_path) == 10:
    img_path= '/content/drive/MyDrive/Proyecto_EL5206/jpg/'+ str(img_path)
  #There is an interpolation method to match the source size with the target size image loaded in PIL (Python Imaging Library)
  img = image.load_img(img_path,color_mode='rgb',target_size=(224, 224))
  # Converts a PIL Image to 3D Numy Array
  x = image.img_to_array(img)
  # Adding the fouth dimension, for number of images
  x = np.expand_dims(x, axis=0)
  #mean centering with respect to Image
  x = preprocess_input(x)
  features = model.predict(x)

  return features
model = VGG16(weights='imagenet')

Separamos las imagenes en consulta y buscador en la base de datos

In [22]:
import os
imput_path = '/content/drive/MyDrive/Proyecto_EL5206/jpg'

nombres_img = os.listdir(imput_path)
nombres_img= sorted(nombres_img)
global img_database
global img_query

img_query= []
img_database= []
for nombre in nombres_img:
  if nombre[5] == '0':
    img_query.append(nombre)
  else:
     img_database.append(nombre)



p1

In [28]:
def vector_caracteristicas(img,extractor, model=model):
  if str(extractor) == 'clasico':
    vec  = handcrafted(img)
  elif str(extractor) == 'CNN' or str(extractor) == 'cnn':
    vec = cnn_vgg(img, model)
  else:
    print('Por favor intente nuevamente, ingresando si el extractor es CNN o clasico')
  return vec

##Medidas de distancia

In [24]:
#Definimos la distancia euclidiana USAR ESTA
def similarity_euclid(v1, v2):
  return sqrt(sum(pow(a-b,2) for a, b in zip(v1, v2)))

#Definimos la distancia coseno
def square_rooted(v1):
   return round(sqrt(sum([a*a for a in v1])),3)
  
def similarity_cos(v1, v2):
 numerator = sum(a*b for a,b in zip(v1,v2))
 denominator = square_rooted(v1)*square_rooted(v2)
 return round(numerator/float(denominator),3)

## 3.Ordenar resultados por relevancia: 

In [25]:
#Esquema comparación
def Esquema_comparacion(extractor):
  esquema= []
  for img in img_query:
    v_q= vector_caracteristicas(img,extractor)
    lista_comp= []
    for img_d in img_database:
      v_d= vector_caracteristicas(img_d,extractor)
      d= similarity_euclid(v_q,v_d)
      lista_comp.append((img_d,d))
    esquema.append((img,lista_comp))
  return esquema

In [29]:
vec_query_clasico=[]
vec_query_CNN=[]
for img in img_query:
  v_q= vector_caracteristicas(img,"clasico")
  vec_query_clasico.append([img,v_q])
  v_q= vector_caracteristicas(img,"CNN")
  vec_query_CNN.append([img,v_q])



1/1 [==============================] - 1s 581ms/step


In [30]:
import pickle
with open("vec_query_CNN.pickle", "wb") as f:
    pickle.dump(vec_query_CNN, f)

In [31]:
import pickle
with open("vec_query_clasico.pickle", "wb") as f:
    pickle.dump(vec_query_clasico, f)